In [0]:
#1. Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы
import pandas as pd
movies = pd.read_csv('u.item.csv', delimiter='|', encoding='latin-1', names = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])
ratings = pd.read_csv('u.data.csv', delimiter='\t', encoding='latin-1', names = ['user_id', 'item_id', 'rating', 'timestamp'])

In [420]:
movies.head(5)

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [421]:
ratings.head(5)

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [422]:
#2. Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок
#варинат 1 (получается пользователь 405 поставил больше всех оценок 737)
ratings['user_id'].value_counts(ascending=False)
#варинат 2 (более сложный)
ratings.groupby('user_id').agg('count').loc[:,['rating']].sort_values(by='rating', ascending=False)

,rating
user_id,
405,737
655,685
13,636
450,540
276,518
...,...
685,20
475,20
36,20


In [423]:
#3.Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь
df = []
df = ratings[ratings['user_id'] == 405].join(movies.set_index('movie_id'), how='inner', on='item_id')
df

,user_id,item_id,rating,timestamp,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
12276,405,56,4,885544911,Pulp Fiction (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Pulp%20Fictio...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
12383,405,592,1,885548670,True Crime (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?True%20Crime%...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
12430,405,1582,1,885548670,T-Men (1947),01-Jan-1947,NaN,http://us.imdb.com/M/title-exact?T-Men%20(1947),0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
12449,405,171,1,885549544,Delicatessen (1991),01-Jan-1991,NaN,http://us.imdb.com/M/title-exact?Delicatessen%...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
12460,405,580,1,885547447,"Englishman Who Went Up a Hill, But Came Down a...",01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Englishman%20...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98956,405,375,1,885546835,Showgirls (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Showgirls%20(...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
98978,405,445,4,885548435,"Body Snatcher, The (1945)",01-Jan-1945,NaN,http://us.imdb.com/M/title-exact?Body%20Snatch...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
99148,405,1246,1,885547735,Bushwhacked (1995),01-Jan-1995,NaN,http://us.imdb.com/Title?Bushwhacked+(1995/I),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
99465,405,196,1,885546112,Dead Poets Society (1989),01-Jan-1989,NaN,http://us.imdb.com/M/title-exact?Dead%20Poets%...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [424]:
# 4. Добавьте к датафрейму из задания 3 столбца
# 4.1 По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет
#сделано в прошлом задании с помощью join
df['relese_year'] = df.apply(lambda row: int(row['release_date'][-4:]), axis=1)
df

,user_id,item_id,rating,timestamp,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,relese_year
12276,405,56,4,885544911,Pulp Fiction (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Pulp%20Fictio...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1994
12383,405,592,1,885548670,True Crime (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?True%20Crime%...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1995
12430,405,1582,1,885548670,T-Men (1947),01-Jan-1947,NaN,http://us.imdb.com/M/title-exact?T-Men%20(1947),0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1947
12449,405,171,1,885549544,Delicatessen (1991),01-Jan-1991,NaN,http://us.imdb.com/M/title-exact?Delicatessen%...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1991
12460,405,580,1,885547447,"Englishman Who Went Up a Hill, But Came Down a...",01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Englishman%20...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98956,405,375,1,885546835,Showgirls (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Showgirls%20(...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1995
98978,405,445,4,885548435,"Body Snatcher, The (1945)",01-Jan-1945,NaN,http://us.imdb.com/M/title-exact?Body%20Snatch...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1945
99148,405,1246,1,885547735,Bushwhacked (1995),01-Jan-1995,NaN,http://us.imdb.com/Title?Bushwhacked+(1995/I),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1995
99465,405,196,1,885546112,Dead Poets Society (1989),01-Jan-1989,NaN,http://us.imdb.com/M/title-exact?Dead%20Poets%...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1989


In [425]:
#4.2. столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей
tmp_rat_cnt = ratings.groupby('item_id').agg('count').loc[:,['rating']]
tmp_rat_cnt.rename({'rating': 'rating_cnt'}, axis=1, inplace=True)
tmp_rat_sum = ratings.groupby('item_id').agg('sum').loc[:,['rating']]
tmp_rat_sum.rename({'rating': 'rating_sum'}, axis=1, inplace=True)
tmp_rat = tmp_rat_cnt.join(tmp_rat_sum, how='inner')
tmp_rat.reset_index()

,item_id,rating_cnt,rating_sum
0,1,452,1753
1,2,131,420
2,3,90,273
3,4,209,742
4,5,86,284
...,...,...,...
1677,1678,1,1
1678,1679,1,3
1679,1680,1,2
1680,1681,1,3


In [426]:
df = df.join(tmp_rat, how = 'inner', on='item_id')
df

,user_id,item_id,rating,timestamp,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,relese_year,rating_cnt,rating_sum
12276,405,56,4,885544911,Pulp Fiction (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Pulp%20Fictio...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1994,394,1600
12383,405,592,1,885548670,True Crime (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?True%20Crime%...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1995,9,30
12430,405,1582,1,885548670,T-Men (1947),01-Jan-1947,NaN,http://us.imdb.com/M/title-exact?T-Men%20(1947),0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1947,1,1
12449,405,171,1,885549544,Delicatessen (1991),01-Jan-1991,NaN,http://us.imdb.com/M/title-exact?Delicatessen%...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1991,65,252
12460,405,580,1,885547447,"Englishman Who Went Up a Hill, But Came Down a...",01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Englishman%20...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1995,32,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98956,405,375,1,885546835,Showgirls (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Showgirls%20(...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1995,23,45
98978,405,445,4,885548435,"Body Snatcher, The (1945)",01-Jan-1945,NaN,http://us.imdb.com/M/title-exact?Body%20Snatch...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1945,22,77
99148,405,1246,1,885547735,Bushwhacked (1995),01-Jan-1995,NaN,http://us.imdb.com/Title?Bushwhacked+(1995/I),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1995,7,11
99465,405,196,1,885546112,Dead Poets Society (1989),01-Jan-1989,NaN,http://us.imdb.com/M/title-exact?Dead%20Poets%...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1989,251,983


In [429]:
# 5,6 - делим данные и тренируем модель
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
x = df[['Action',	'Adventure',	'Animation',	'Childrens',	'Comedy',	'Crime',	'Documentary',	'Drama',	'Fantasy',	'Film-Noir',	'Horror',	'Musical',	'Mystery',	'Romance',	'Sci-Fi',	'Thriller',	'War',	'Western', 'relese_year', 'rating_cnt',	'rating_sum']]
y = df[['rating']]
#делим данные на тренировочные и тестовые
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)
lr = LinearRegression()

#обучаем модель
lr.fit(x_train, y_train)
print(lr.coef_, lr.intercept_)

[[-4.98341093e-02 -2.92454544e-01 -2.28209931e-01  4.98875379e-01
   2.37648043e-01 -9.53027551e-03  4.84420995e-01  2.96563736e-01
  -1.39733402e-01 -4.89600147e-01  1.35226202e-01  8.97138461e-01
   4.66364604e-01 -6.80768610e-04 -2.13424577e-01  1.48821653e-01
  -4.84257584e-01  3.20362378e-02 -9.64275736e-03  1.79215126e-02
  -2.31718409e-03]] [20.14240568]


In [430]:
# 7 Оцените качество модели на x_test, y_test при помощи метрик для задачи регрессии
#смотрим результаты работы моедли на тестовых даных
import numpy as np
y_predict = lr.predict(x_test)
lr_mse = mean_squared_error(y_test, y_predict)
#оценим качество предсказания модели
lr_mae = mean_absolute_error(y_test, y_predict)
lr_r2 = r2_score(y_test, y_predict)
print('Mean Square error: ',lr_mse, '\nMean Absolute error: ', lr_mae, '\nR^2: ', lr_r2)

Mean Square error:  1.1229159218302458 
Mean Absolute error:  0.823000612259093 
R^2:  0.3613991496580722


In [431]:
from sklearn.svm import SVR
import numpy as np
svr = SVR(gamma = 'scale', C=1.0, epsilon=0.2)
svr.fit(np.array(x_train), np.ravel(np.array(y_train)))
y_pred_svr = svr.predict(x_test)
svr_mse = mean_squared_error(y_test, y_pred_svr)
svr_mae = mean_absolute_error(y_test, y_pred_svr)
svr_r2 = r2_score(y_test, y_pred_svr)
print('Mean Square error: ',svr_mse, '\nMean Absolute error: ', svr_mae, '\nR^2: ', svr_r2)

Mean Square error:  1.1100752292778249 
Mean Absolute error:  0.6707575454292658 
R^2:  0.36870163511004606


In [432]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(np.array(x_train), np.ravel(np.array(y_train)))
y_pred_dtr = dtr.predict(x_test)
dtr_mse = mean_squared_error(y_test, y_pred_dtr)
dtr_mae = mean_absolute_error(y_test, y_pred_dtr)
dtr_r2 = r2_score(y_test, y_pred_dtr)
print('Mean Square error: ',dtr_mse, '\nMean Absolute error: ', dtr_mae, '\nR^2: ', dtr_r2)

Mean Square error:  2.0945945945945947 
Mean Absolute error:  0.8513513513513513 
R^2:  -0.1911932703292134


Вывод: качество модели низкое, так как коэффициент детерминации r2 составляет максимум 37% для модели линейной регрессии, что значительно ниже 50%. вывод: необходимо улучшать модель. Например тренировать модель на большем количестве данных

In [0]:
# Sparc
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [303]:
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz

spark-2.4.5-bin-hadoop2.7/
spark-2.4.5-bin-hadoop2.7/licenses/
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-jtransforms.html
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-zstd.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-zstd-jni.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-xmlenc.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-vis.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-spire.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-sorttable.js.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-slf4j.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-scopt.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-scala.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-sbt-launch-lib.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-respond.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-reflectasm.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-pyrolite.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-py4j.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-protobuf.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-pmml-model

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
ratings = spark.read.option('header', True).option('delimiter', '\\t').option('inferSchema', True).csv('u.data_sp.csv')
movies = spark.read.option('header', True).option('delimiter', '|').option('inferSchema', True).csv('u.item_sp.csv')


In [437]:
movies.show(5)
ratings.show(5)

+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|      movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1| Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|        1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2| GoldenEye (1995)| 01-Jan-1995|              null|

In [438]:
#9. Средствами спарка вывести среднюю оценку для каждого фильма
ratgrp = ratings.groupBy('item_id')
ratgrp = ratgrp.avg('rating').select('item_id', 'avg(rating)')
ratgrp.join(movies, on=ratgrp['item_id'] == movies['movie_id'], how = 'inner').select('movie_title', 'avg(rating)').show()

+--------------------+------------------+
|         movie_title|       avg(rating)|
+--------------------+------------------+
|It's a Wonderful ...| 4.121212121212121|
|Courage Under Fir...|3.6108597285067874|
|Secret of Roan In...| 3.859154929577465|
|Ghost and the Dar...|          3.203125|
|Convent, The (Con...|               2.5|
|  Bulletproof (1996)| 3.204081632653061|
|  Double Team (1997)| 2.230769230769231|
|Duoluo tianshi (1...|3.1666666666666665|
|   Full Speed (1996)|             3.125|
|     Liebelei (1933)|               1.0|
|Butcher Boy, The ...|               4.0|
|Man Without a Fac...|3.5441176470588234|
|Angels in the Out...| 2.923076923076923|
|  Money Train (1995)| 2.511627906976744|
|Amityville: Dollh...|               1.0|
|       Sirens (1994)| 2.983050847457627|
|Jungle2Jungle (1997)|2.4393939393939394|
|Fire Down Below (...|2.9318181818181817|
|Anne Frank Rememb...| 3.857142857142857|
|Truman Show, The ...| 2.909090909090909|
+--------------------+------------

In [439]:
#10. Посчитайте средствами спарка среднюю оценку для каждого жанра
unidf = ratings.join(movies, on=ratings['item_id'] == movies['movie_id'], how = 'inner')
unidf.where(unidf['Action'] == 1).groupBy('Action').avg('rating').show()
unidf.where(unidf['Adventure'] == 1).groupBy('Adventure').avg('rating').show()
unidf.where(unidf['Animation'] == 1).groupBy('Animation').avg('rating').show()
unidf.where(unidf['Childrens'] == 1).groupBy('Childrens').avg('rating').show()
unidf.where(unidf['Comedy'] == 1).groupBy('Comedy').avg('rating').show()
unidf.where(unidf['Crime'] == 1).groupBy('Crime').avg('rating').show()
unidf.where(unidf['Documentary'] == 1).groupBy('Documentary').avg('rating').show()
unidf.where(unidf['Drama'] == 1).groupBy('Drama').avg('rating').show()
unidf.where(unidf['Fantasy'] == 1).groupBy('Fantasy').avg('rating').show()
unidf.where(unidf['Film-Noir'] == 1).groupBy('Film-Noir').avg('rating').show()
unidf.where(unidf['Horror'] == 1).groupBy('Horror').avg('rating').show()
unidf.where(unidf['Musical'] == 1).groupBy('Musical').avg('rating').show()
unidf.where(unidf['Mystery'] == 1).groupBy('Mystery').avg('rating').show()
unidf.where(unidf['Romance'] == 1).groupBy('Romance').avg('rating').show()
unidf.where(unidf['Sci-Fi'] == 1).groupBy('Sci-Fi').avg('rating').show()
unidf.where(unidf['Thriller'] == 1).groupBy('Thriller').avg('rating').show()
unidf.where(unidf['War'] == 1).groupBy('War').avg('rating').show()
unidf.where(unidf['Western'] == 1).groupBy('Western').avg('rating').show()

+------+-----------------+
|Action|      avg(rating)|
+------+-----------------+
|     1|3.480245417953027|
+------+-----------------+

+---------+-----------------+
|Adventure|      avg(rating)|
+---------+-----------------+
|        1|3.503526503308369|
+---------+-----------------+

+---------+------------------+
|Animation|       avg(rating)|
+---------+------------------+
|        1|3.5766990291262135|
+---------+------------------+

+---------+------------------+
|Childrens|       avg(rating)|
+---------+------------------+
|        1|3.3532442216652742|
+---------+------------------+

+------+------------------+
|Comedy|       avg(rating)|
+------+------------------+
|     1|3.3940734781442745|
+------+------------------+

+-----+------------------+
|Crime|       avg(rating)|
+-----+------------------+
|    1|3.6322780881440098|
+-----+------------------+

+-----------+------------------+
|Documentary|       avg(rating)|
+-----------+------------------+
|          1|3.6728232189

In [440]:
#11 получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок)
# ТОП5 популярных фильмов
unidf.groupBy('movie_title').count().sort('count', ascending=False).show(5)
# 5 самых непопулярных фильмов
unidf.groupBy('movie_title').count().sort('count', ascending=True).show(5)

+--------------------+-----+
|         movie_title|count|
+--------------------+-----+
|    Star Wars (1977)|  583|
|      Contact (1997)|  509|
|        Fargo (1996)|  508|
|Return of the Jed...|  507|
|    Liar Liar (1997)|  485|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
|         movie_title|count|
+--------------------+-----+
|    Fear, The (1995)|    1|
|Vie est belle, La...|    1|
|Modern Affair, A ...|    1|
|       Target (1995)|    1|
|JLG/JLG - autopor...|    1|
+--------------------+-----+
only showing top 5 rows

